In [1]:
import tensorflow as tf
model = tf.keras.models.load_model('toxic_comment_classification.h5')

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
test_df = pd.read_csv('test.csv')


In [6]:
y_test = pd.read_csv('test_labels.csv')

In [7]:
X_test = test_df['comment_text']

In [8]:
X_test

0         Yo bitch Ja Rule is more succesful then you'll...
1         == From RfC == \n\n The title is fine as it is...
2         " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3         :If you have a look back at the source, the in...
4                 I don't anonymously edit articles at all.
                                ...                        
153159    . \n i totally agree, this stuff is nothing bu...
153160    == Throw from out field to home plate. == \n\n...
153161    " \n\n == Okinotorishima categories == \n\n I ...
153162    " \n\n == ""One of the founding nations of the...
153163    " \n :::Stop already. Your bullshit is not wel...
Name: comment_text, Length: 153164, dtype: object

In [9]:

y_test =y_test[y_test.columns[2:]].values
y_test

array([[-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       ...,
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]], dtype=int64)

In [10]:
X_test.shape

(153164,)

In [11]:
y_test.shape

(153164, 5)

In [12]:
MAX_FEATURES = 200000

In [13]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                              output_sequence_length = 1800,
                              output_mode = 'int')

In [14]:
vectorizer.adapt(X_test.values)

In [15]:
vectorizer('I am human and I live in this world ')

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([  9,  90, 651, ...,   0,   0,   0], dtype=int64)>

In [16]:
vectorized_text_test = vectorizer(X_test.values)

In [17]:
dataset_test = tf.data.Dataset.from_tensor_slices((vectorized_text_test,y_test))
dataset_test = dataset_test.cache()
dataset_test = dataset_test.shuffle(160000)
dataset_test = dataset_test.batch(16)
dataset_test = dataset_test.prefetch(8)

In [18]:
batch_X_test,batch_y_test = dataset_test.as_numpy_iterator().next()

In [19]:
batch_X_test.shape

(16, 1800)

In [20]:
batch_y_test.shape

(16, 5)

In [22]:
test_data = dataset_test.take(int(len(dataset_test)*1))

In [23]:
batch_X_test , batch_y_test = test_data.as_numpy_iterator().next()

In [24]:
(model.predict(batch_X_test) >0.5).astype(int)

1/1 [==============================] - 3s 3s/step


array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [26]:
batch_y_test

array([[ 0,  1,  0,  1,  0],
       [-1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1]], dtype=int64)

### MODEL EVALUATION

In [27]:
from tensorflow.keras.metrics import Precision , Recall , CategoricalAccuracy

In [30]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [31]:
for batch in test_data.as_numpy_iterator():
    X_true_test,y_true_test = batch # unpacking the batch
    yhat_test = model.predict(X_true_test)  #predictions
    
    y_true_test = y_true_test.flatten() #flatten redictions
    y_hat_test = yhat_test.flatten()
    
    pre.update_state(y_true_test,y_hat_test)
    re.update_state(y_true_test,y_hat_test)
    acc.update_state(y_true_test,y_hat_test)

1/1 [==============================] - 0s 394ms/step


ValueError: Shapes (96,) and (80,) are incompatible

In [ ]:
print(f'precision:{pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}')